In [ ]:
#
#    Notebook de cours MAP412 - Chapitre 5 - M. Massot 2020-2021 - Ecole polytechnique
#    ----------   
#    Méthodes de résolution de systèmes linéaires itératives 
#    stationnaires et instationnaires
#    
#    Auteurs : L. Séries et M. Massot - (C) 2021
#    

In [ ]:
import numpy as np
from scipy.sparse import diags
from scipy.sparse.linalg import inv
import plotly.graph_objects as go

# Méthodes stationnaires et instationnaires

On souhaite résoudre le problème elliptique constitué par l'équation de Poisson soumise à des conditions aux limites de type de Dirichlet :

$$
\left\{
\begin{aligned}
-\Delta u(x) & =  b(x) \quad \text{dans} \; \Omega = [0,1] \\
        u(0) & =  0 \quad \text{sur}  \;  \partial \Omega
\end{aligned}
\right.
$$

## Méthode de Jacobi

In [ ]:
def jacobi(a, b, max_iter=100000, eps=1.e-6):
    xk = np.zeros(b.size)
    norm_b = np.linalg.norm(b)

    lpu = diags([a.diagonal(-1), a.diagonal(1)], [-1, 1])
    inv_d = diags([1/a.diagonal()], [0])

    ite_mat = -inv_d*lpu

    for i in range(max_iter):
        xk = ite_mat.dot(xk) + inv_d.dot(b)
        rk = a.dot(xk) - b
        norm_rk = np.linalg.norm(rk)
        if (norm_rk/norm_b < eps): break

    print(f"  Nombre d'itérations = {i}")
    print(f"  ||A.xk - b|| / ||b|| = {norm_rk/norm_b}")

    return xk

In [ ]:
# nb of points without boundaries
nx = 100
dx = 1/(nx+1)
diagonals = [np.repeat(2/(dx*dx), nx), np.repeat(-1/(dx*dx), nx-1), np.repeat(-1/(dx*dx), nx-1)]
a = diags(diagonals, [0, -1, 1])

b = np.ones(nx)

print("\nRésolution par la méthode de Jacobi")
u = jacobi(a, b, max_iter=30000)
u = np.concatenate((np.array([0]), u, np.array([0])))

# show solution
#x = np.linspace(0+dx, 1-dx, nx)
x = np.linspace(0, 1, nx+2)
fig = go.Figure(go.Scatter(x=x, y=u, name='u', mode="markers"))
fig.show()

## Méthode de Gauss-Seidel

In [ ]:
def gauss_seidel(a, b, max_iter=100000, eps=1.e-6):
    xk = np.zeros(b.size)
    norm_b = np.linalg.norm(b)

    dpl = diags([a.diagonal(-1), a.diagonal(0)], [-1, 0])
    inv_dpl = inv(dpl.tocsc())
    u = diags([a.diagonal(1)], [1])
    ite_mat = -inv_dpl*u

    for k in range(max_iter):
        xk = ite_mat.dot(xk) + inv_dpl.dot(b)
        rk = a.dot(xk) - b
        norm_rk = np.linalg.norm(rk)
        if (norm_rk/norm_b < eps): break

    print(f"  Nombre d'itérations = {k+1}")
    print(f"  ||A.xk - b|| / ||b|| = {norm_rk/norm_b}")

    return xk

In [ ]:
# nb of points without boundaries
nx = 100
dx = 1/(nx+1)
diagonals = [np.repeat(2/(dx*dx), nx), np.repeat(-1/(dx*dx), nx-1), np.repeat(-1/(dx*dx), nx-1)]
a = diags(diagonals, [0, -1, 1])

b = np.ones(nx)

print("\nRésolution par la méthode du Gauss-Seidel")
u = gauss_seidel(a, b, max_iter=30000)
u = np.concatenate((np.array([0]), u, np.array([0])))

# show solution
#x = np.linspace(0+dx, 1-dx, nx)
x = np.linspace(0, 1, nx+2)
fig = go.Figure(go.Scatter(x=x, y=u, name='u', mode="markers"))
fig.show()

## Méthode du gradient conjugué

In [ ]:
def conjugate_gradient(a, b, eps=1.e-6):
    xk = np.zeros(b.size)
    norm_b = np.linalg.norm(b)

    rk = b - a.dot(xk)
    pk = rk
    rkm1 = rk

    for k in range(b.size):
        apk = a.dot(pk)
        alpha = np.dot(rk,rk) / np.dot(pk, apk)
        xk = xk + alpha*pk
        rk = rk - alpha*apk
        norm_rk = np.linalg.norm(rk)
        #print(k, np.linalg.norm(rk))
        if norm_rk/norm_b < eps: break
        beta = np.dot(rk,rk) / np.dot(rkm1, rkm1)
        pk = rk + beta*pk
        rkm1 = rk

    print(f"  Nombre d'itérations = {k+1}")
    print(f"  ||A.xk - b|| / ||b|| = {norm_rk/norm_b}")

    return xk

In [ ]:
# nb of points without boundaries
nx = 100
dx = 1/(nx+1)
diagonals = [np.repeat(2/(dx*dx), nx), np.repeat(-1/(dx*dx), nx-1), np.repeat(-1/(dx*dx), nx-1)]
a = diags(diagonals, [0, -1, 1])

b = np.ones(nx)
print("\nRésolution par la méthode du gradient conjugué")
u = conjugate_gradient(a, b)
u = np.concatenate((np.array([0]), u, np.array([0])))

# show solution
#x = np.linspace(0+dx, 1-dx, nx)
x = np.linspace(0, 1, nx+2)
fig = go.Figure(go.Scatter(x=x, y=u, name='u', mode="markers"))
fig.show()